This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [8]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'c'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [9]:
# First, import the relevant modules
import requests
import collections


In [152]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

res = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-06-25&api_key=' + API_KEY)
stockdict = res.json()
stockdict

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2020-06-25',
  'end_date': '2020-06-25',
  'frequency': 'daily',
  'data': [['2020-06-25',
    86.25,
    88.25,
    84.8,
    87.8,
    None,
    118166.0,
    10302946.05,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [56]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
stockdict['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [162]:
# API Request for 2017
start_date_string='?start_date=' + '2017-01-01'
end_date_string = '&end_date=' + '2017-12-31'
api_string = '&api_key=' + API_KEY
res = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json' + start_date_string + end_date_string + api_string)

#Convert to dict filetype and inspect structure
stockdict = res.json()
stockdict['dataset_data'].keys()
columns = stockdict['dataset_data']['column_names']
print(columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [276]:
# Save list containing required data into variable
stock_data = stockdict['dataset_data']['data']

# Get list positions of Open, High, Low, Close, and Traded Volume
open_x = columns.index('Open')
high_x = columns.index('High')
low_x = columns.index('Low')
close_x = columns.index('Close')
volume_x = columns.index('Traded Volume')

# Un-nest lists into separate variable names
# Initialize empty lists, then append each nested value to a separate list
Open = []
High = []
Low = []
Close = []
Volume = []

for item in stock_data:
    Open.append(item[open_x])
    High.append(item[high_x])
    Low.append(item[low_x])
    Close.append(item[close_x])
    Volume.append(item[volume_x])
    
# 'Open' has NoneTypes. Filter these out of the list in order to take max and min.
Open = list(filter(None, Open))

# Calculate what the highest and lowest opening prices were for the stock in this period.
print('Highest and Lowest Opening Prices: ')
print(max(Open))
print(min(Open))

# What was the largest change in any one day (based on High and Low price)?
# (Using list comprehension to create lists of differences between daily high and low prices, then compare greatest increase to greatest decrease)
day_rise = [High - Low for High, Low in zip(High, Low)]
day_fall = [Low - High for High, Low in zip(High, Low)]
print('Largest Change in Any One Day: ')
if abs(max(day_rise)) > abs(min(day_fall)):
    print(max(day_rise))
else:
    print(min(day_fall))

# What was the largest change between any two days (based on Closing Price)?
# (Initialize empty list for two_day_change and loop through Close list to append difference between Close[index] and Close[index + 2] for all feasible index numbers)
two_day_change = []

index = 0
print('Maximum Two-Day Change in Close Price: ')
while (index < len(Close)-2):
    two_day_change.append(Close[index+2]-Close[index])
    index +=1
if max(two_day_change) > abs(min(two_day_change)):
    print(max(two_day_change))
else:
    print(min(two_day_change))

# What was the average daily trading volume during this year?
print('Average Daily Trading Volume: ')
print(sum(Volume)/len(Volume))

# (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Highest and Lowest Opening Prices: 
53.11
34.0
Largest Change in Any One Day: 
-2.8100000000000023
Maximum Two-Day Change in Close Price: 
3.1499999999999986
Average Daily Trading Volume: 
89124.33725490196
